In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mtp
import seaborn as sns
import warnings
import plotly.express as px
warnings.filterwarnings("ignore")

In [ ]:
artist_data = pd.read_csv("data_by_artist.csv")
data = pd.read_csv("data.csv")

In [ ]:
artist_data.info()

In [ ]:
artist_data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
features = ['acousticness','danceability', 'energy','instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence', 'tempo', 'duration_ms', 'key']

In [ ]:
numeric_song_data = data[features]
numeric_song_data

In [ ]:
from sklearn.preprocessing import StandardScaler
#scaling data
song_scaler = StandardScaler()
scaled_song_data = song_scaler.fit_transform(numeric_song_data)

#clusteriong the scaled data
from sklearn.cluster import KMeans
song_cluster = KMeans(n_clusters = 20)
song_cluster.fit(scaled_song_data)


In [ ]:
# labeling song clusters
song_cluster_labels = song_cluster.labels_

# addinng cluster lables to the datset
data['clusters'] = song_cluster_labels

# reducing the dimensionality for graph plotting.
# as data has linear relation instead of TSNE we use PCA

from sklearn.decomposition import PCA

song_reducer = PCA(n_components = 2)
reduced_song_data = song_reducer.fit_transform(scaled_song_data)

In [ ]:
new_song_dataset = pd.DataFrame(columns = ['x-axis', 'y-axis'], data = reduced_song_data)
new_song_dataset['name'] = data['name']
new_song_dataset['clusters'] = data['clusters']

In [ ]:
song_graph = px.scatter(new_song_dataset, x = 'x-axis', y = 'y-axis', color = 'clusters', hover_data = ['x-axis', 'y-axis', 'clusters'])
song_graph.show()

# Recommending songs.
    If the input is song name.

In [ ]:
def ArtistBasedRecommender1(song_name):
#     finding the cluster of the song
    song_cluster = data.loc[data['name'] == song_name, 'clusters'].values[0]
#     finding the artist of the song
    song_artist = data.loc[data['name'] == song_name, 'artists'].values[0]
#     finding similar cluster songs
    similar_cluster_songs = data.loc[data['clusters'] == song_cluster]
#     finding the similar artist songs from similar clustered songs
    similar_artist_songs = similar_cluster_songs.loc[similar_cluster_songs['artists'] == song_artist]
    recommending_songs = similar_artist_songs.loc[similar_artist_songs['name'] != song_name]
    top_recommending_songs = recommending_songs.nlargest(5, 'popularity')
    
    ls = []
    
    for index,row in top_recommending_songs.iterrows():
        name_of_song = row['name']
        ls.append(name_of_song)
    return ls
   

In [ ]:
best_songs = ArtistBasedRecommender1('Ace')
for i in best_songs:
    print(i)

# Recommending songs
    If the input is artist name.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def ArtistBasedRecommender2(song_artist):

    artist_songs = data.loc[data['artists'] == song_artist]
    
    top_recommending_songs = artist_songs.nlargest(5, 'popularity')
    
    for index,row in top_recommending_songs.iterrows():
        song_name = row['name']
        print(song_name)


In [ ]:
best_songs = ArtistBasedRecommender2("['Eminem']")
best_songs